In [1]:
import pandas as pd
import duckdb

In [2]:
# Return Speed in km/h from distance and time
def to_km_h(distance, time):
    if time != 0:
        return distance/time * (3600/1000)
    return 0
# Return time in secodnes from distance and speed
def to_time(distance, speed):
    if speed != 0:
        return distance/speed * (3600/1000)
    return 0

# Compute duration between two timestamps
def duration(time1, time2):
    hour1,minute1,second1 = [int(x) for x in time1.split(':')]
    hour2,minute2,second2 = [int(x) for x in time2.split(':')]
    if hour1 < 3 and hour2 > 20:
        hour1 = hour1 + 24
    if hour1 > 20 and hour2 < 3:
        hour2 = hour2 + 24
    return hour1*3600 + minute1*60 + second1 - hour2*3600 - minute2*60 - second2

In [3]:
# Reframe the schedule dataframe where each line was a node to a dataframe where each line is an edge
def reframe_df(df):
    col = {'DATDEP': [], 'TRAIN_NO': [], 'RELATION': [], 'REAL_TIME': [], 'PLANNED_TIME': [], 'DELAY_DEP':[],
           'NODE_1': [], 'PTCAR_1': [], 'LINE_NO_1': [], 'PTCAR_ID_1': [],
           'NODE_2': [], 'PTCAR_2': [], 'LINE_NO_2': [], 'PTCAR_ID_2': [], 'HOUR': []}
    
    for ind in df.index:
        if ind+1 == len(df.index):
            break
        if df['TRAIN_NO'][ind] == df['TRAIN_NO'][ind+1] and df['DATDEP'][ind] == df['DATDEP'][ind+1]:
            col['DATDEP'].append(df['DATDEP'][ind])
            col['TRAIN_NO'].append(df['TRAIN_NO'][ind])
            col['RELATION'].append(df['RELATION'][ind])
            col['REAL_TIME'].append(duration(df['REAL_TIME_DEP'][ind+1], df['REAL_TIME_DEP'][ind]))
            col['PLANNED_TIME'].append(duration(df['PLANNED_TIME_DEP'][ind+1], df['PLANNED_TIME_DEP'][ind]))
            col['DELAY_DEP'].append(duration(df['REAL_TIME_DEP'][ind], df['PLANNED_TIME_DEP'][ind]))
            col['NODE_1'].append(df['PTCAR_LG_NM_NL'][ind]+'_'+df['LINE_NO_DEP'][ind])
            col['PTCAR_1'].append(df['PTCAR_LG_NM_NL'][ind])
            col['LINE_NO_1'].append(df['LINE_NO_DEP'][ind])
            col['PTCAR_ID_1'].append(df['PTCAR_NO'][ind])
            col['NODE_2'].append(df['PTCAR_LG_NM_NL'][ind+1]+'_'+df['LINE_NO_DEP'][ind+1])
            col['PTCAR_2'].append(df['PTCAR_LG_NM_NL'][ind+1])
            col['LINE_NO_2'].append(df['LINE_NO_DEP'][ind+1])
            col['PTCAR_ID_2'].append(df['PTCAR_NO'][ind+1])
            col['HOUR'].append(int(df['REAL_TIME_DEP'][ind].split(':')[0]))
        
    df = pd.DataFrame(data=col)
    
    return df


In [4]:
def reframe_df_duckdb(df):
    sql = """
        WITH sq AS (
            SELECT
                strptime(DATDEP, '%d%b%Y') AS DATDEP,
                TRAIN_NO,
                RELATION,
                TRAIN_SERV,
                PTCAR_NO,
                LINE_NO_DEP,
                CAST(strptime(REAL_DATE_ARR,    '%d%b%Y') AS DATE) + CAST(strptime(REAL_TIME_ARR,    '%-H:%M:%S') AS TIME) AS REAL_DT_ARR,
                CAST(strptime(REAL_DATE_DEP,    '%d%b%Y') AS DATE) + CAST(strptime(REAL_TIME_DEP,    '%-H:%M:%S') AS TIME) AS REAL_DT_DEP,
                CAST(strptime(PLANNED_DATE_ARR, '%d%b%Y') AS DATE) + CAST(strptime(PLANNED_TIME_ARR, '%-H:%M:%S') AS TIME) AS PLANNED_DT_ARR,
                CAST(strptime(PLANNED_DATE_DEP, '%d%b%Y') AS DATE) + CAST(strptime(PLANNED_TIME_DEP, '%-H:%M:%S') AS TIME) AS PLANNED_DT_DEP,
                DELAY_ARR,
                DELAY_DEP,
                PTCAR_LG_NM_NL,
                LINE_NO_ARR,
            FROM
                df
        ), shifted AS (
            SELECT
                *,
                LEAD(PTCAR_LG_NM_NL) OVER w AS NEXT_PTCAR_LG_NM_NL,
                LEAD(LINE_NO_ARR) OVER w AS NEXT_LINE_NO_ARR,
                LEAD(PTCAR_NO) OVER w AS NEXT_PTCAR_NO,
                LEAD(LINE_NO_DEP) OVER w AS NEXT_LINE_NO_DEP,
                LEAD(REAL_DT_ARR) OVER w AS NEXT_REAL_DT_ARR,
                LEAD(REAL_DT_DEP) OVER w AS NEXT_REAL_DT_DEP,
                LEAD(PLANNED_DT_ARR) OVER w AS NEXT_PLANNED_DT_ARR,
                LEAD(PLANNED_DT_DEP) OVER w AS NEXT_PLANNED_DT_DEP,
            FROM
                sq
            WINDOW w AS (PARTITION BY DATDEP, TRAIN_NO ORDER BY COALESCE(REAL_DT_ARR, REAL_DT_DEP), REAL_DT_DEP)
        )
        SELECT
            DATDEP,
            TRAIN_NO,
            RELATION,
            EXTRACT('epoch' FROM (NEXT_REAL_DT_DEP - REAL_DT_DEP)) AS REAL_TIME,
            EXTRACT('epoch' FROM (NEXT_PLANNED_DT_DEP - PLANNED_DT_DEP)) AS PLANNED_TIME,
            EXTRACT('epoch' FROM (REAL_DT_DEP - PLANNED_DT_DEP)) AS DELAY_DEP,
            PTCAR_LG_NM_NL || '_' || LINE_NO_DEP AS NODE_1,
            PTCAR_LG_NM_NL AS PTCAR_1,
            LINE_NO_DEP AS LINE_NO_1,
            PTCAR_NO AS PTCAR_ID_1,
            NEXT_PTCAR_LG_NM_NL || '_' || NEXT_LINE_NO_DEP AS NODE_2,
            NEXT_PTCAR_LG_NM_NL AS PTCAR_2,
            NEXT_LINE_NO_DEP AS LINE_NO_2,
            NEXT_PTCAR_NO AS PTCAR_ID_2,
            EXTRACT('hour' FROM REAL_DT_DEP) AS HOUR,
        FROM
            shifted
        WHERE
            NEXT_PTCAR_NO IS NOT NULL
        ORDER BY
            DATDEP, TRAIN_NO, REAL_DT_DEP
    """
    return duckdb.sql(sql).df()

In [5]:
df = pd.read_csv('Data_raw_punctuality_202302.csv')

In [6]:
# Repair DataFrame
df.loc[pd.isnull(df['PLANNED_TIME_ARR']), 'PLANNED_TIME_ARR'] = df.loc[pd.isnull(df['PLANNED_TIME_ARR']),'PLANNED_TIME_DEP']
df.loc[pd.isnull(df['PLANNED_TIME_DEP']), 'PLANNED_TIME_DEP'] = df.loc[pd.isnull(df['PLANNED_TIME_DEP']),'PLANNED_TIME_ARR']
df.loc[pd.isnull(df['REAL_TIME_ARR']), 'REAL_TIME_ARR'] = df.loc[pd.isnull(df['REAL_TIME_ARR']),'REAL_TIME_DEP']
df.loc[pd.isnull(df['REAL_TIME_DEP']), 'REAL_TIME_DEP'] = df.loc[pd.isnull(df['REAL_TIME_DEP']),'REAL_TIME_ARR']
df.loc[pd.isnull(df['PLANNED_DATE_ARR']), 'PLANNED_DATE_ARR'] = df.loc[pd.isnull(df['PLANNED_DATE_ARR']),'PLANNED_DATE_DEP']
df.loc[pd.isnull(df['PLANNED_DATE_DEP']), 'PLANNED_DATE_DEP'] = df.loc[pd.isnull(df['PLANNED_DATE_DEP']),'PLANNED_DATE_ARR']
df.loc[pd.isnull(df['REAL_DATE_ARR']), 'REAL_DATE_ARR'] = df.loc[pd.isnull(df['REAL_DATE_ARR']),'REAL_DATE_DEP']
df.loc[pd.isnull(df['REAL_DATE_DEP']), 'REAL_DATE_DEP'] = df.loc[pd.isnull(df['REAL_DATE_DEP']),'REAL_DATE_ARR']
df.loc[pd.isnull(df['LINE_NO_DEP']), 'LINE_NO_DEP'] = 'noline'

In [7]:
pd.to_datetime(df['REAL_DATE_DEP'] + ' ' + df['REAL_TIME_DEP'],format='%d%b%Y %H:%M:%S')

0         2023-02-01 06:23:35
1         2023-02-01 06:25:50
2         2023-02-01 06:26:43
3         2023-02-01 06:27:47
4         2023-02-01 06:31:47
                  ...        
1774542   2023-02-28 18:29:48
1774543   2023-02-28 18:31:24
1774544   2023-02-28 18:34:20
1774545   2023-02-28 18:36:11
1774546   2023-02-28 18:45:09
Length: 1774547, dtype: datetime64[ns]

In [8]:
%time df1 = reframe_df(df)

CPU times: user 1min 22s, sys: 695 ms, total: 1min 23s
Wall time: 1min 23s


In [9]:
%time df2 = reframe_df_duckdb(df)

CPU times: user 3.83 s, sys: 2 s, total: 5.83 s
Wall time: 2.39 s


In [10]:
df1.loc[df1['PTCAR_1'] == 'CHARLEROI-CENTRAL',:].groupby(['LINE_NO_1', 'LINE_NO_2', 'PTCAR_2'])['REAL_TIME'].aggregate(['count', 'min', 'median', 'mean', 'max'])

count  min  median        mean   max
LINE_NO_1 LINE_NO_2 PTCAR_2                                                   
124       112       MARCHIENNE-AU-PONT       79  224   343.0  348.772152   773
          124       MARCHIENNE-AU-PONT     1105  231   327.0  328.260633   891
          124A      MARCHIENNE-AU-PONT       21  315   409.0  441.714286   867
          130A      MARCHIENNE-ZONE           3  361   506.0  465.666667   530
          132       JAMIOULX                  5  612   667.0  687.200000   759
124A      112       MARCHIENNE-AU-PONT     1161  163   302.0  298.959518   973
          124A      MARCHIENNE-AU-PONT       82  285   410.0  402.597561   626
          130A      MARCHIENNE-ZONE          11  349   404.0  431.727273   706
          132       JAMIOULX                  5  644   691.0  712.200000   788
124L/2    124L/2    CHARLEROI-FAISCEAU A      3   73    77.0   75.666667    77
          noline    CHARLEROI-FAISCEAU A      1   41    41.0   41.000000    41
130       130       COUILLET               1150   60   151.0  174.144348   585
130A      130A      MARCHIENNE-ZONE         399  293   373.0  397.894737  2806
          132       JAMIOULX                457  584   682.0  698.091904  1005
130C      130C      COUILLET                346  264   369.0  369.471098   579
140/1     140       CHARLEROI-OUEST         467  133   220.0  221.751606   477
          noline    CHARLEROI-OUEST           1  163   163.0  163.000000   163

In [11]:
df2.loc[df2['PTCAR_1'] == 'CHARLEROI-CENTRAL',:].groupby(['LINE_NO_1', 'LINE_NO_2', 'PTCAR_2'])['REAL_TIME'].aggregate(['count', 'min', 'median', 'mean', 'max'])

count   min  median         mean  \
LINE_NO_1 LINE_NO_2 PTCAR_2                                                  
124       112       MARCHIENNE-AU-PONT       79   224   343.0   348.772152   
          124       MARCHIENNE-AU-PONT     1105   231   327.0   328.260633   
          124A      COURCELLES-MOTTE          1  1025  1025.0  1025.000000   
                    MARCHIENNE-AU-PONT       20   315   406.5   439.800000   
          130A      MARCHIENNE-ZONE           3   361   506.0   465.666667   
          132       JAMIOULX                  5   612   667.0   687.200000   
124A      112       MARCHIENNE-AU-PONT     1161   163   302.0   298.959518   
          124A      MARCHIENNE-AU-PONT       82   285   410.0   402.597561   
          130A      MARCHIENNE-ZONE          11   349   404.0   431.727273   
          132       JAMIOULX                  5   644   691.0   712.200000   
124L/2    124L/2    CHARLEROI-FAISCEAU A      3    73    77.0    75.666667   
          noline    CHARLEROI-FAISCEAU A      1    41    41.0    41.000000   
130       124L/2    CHARLEROI-FAISCEAU A      4  -258  -186.5  -194.250000   
                    MARCINELLE                1  -145  -145.0  -145.000000   
          130       COUILLET               1145    60   151.0   174.245415   
130A      130A      MARCHIENNE-ZONE         399   293   373.0   397.894737   
          132       JAMIOULX                457   584   682.0   698.091904   
130C      130C      COUILLET                346   264   369.0   369.471098   
140/1     140       CHARLEROI-OUEST         467   133   220.0   221.751606   
          noline    CHARLEROI-OUEST           1   163   163.0   163.000000   
noline    124A      ROUX                      1     1     1.0     1.000000   
          124L/2    CHARLEROI-FAISCEAU A      1    34    34.0    34.000000   
                    MARCINELLE                2    61    98.5    98.500000   

                                           max  
LINE_NO_1 LINE_NO_2 PTCAR_2                     
124       112       MARCHIENNE-AU-PONT     773  
          124       MARCHIENNE-AU-PONT     891  
          124A      COURCELLES-MOTTE      1025  
                    MARCHIENNE-AU-PONT     867  
          130A      MARCHIENNE-ZONE        530  
          132       JAMIOULX               759  
124A      112       MARCHIENNE-AU-PONT     973  
          124A      MARCHIENNE-AU-PONT     626  
          130A      MARCHIENNE-ZONE        706  
          132       JAMIOULX               788  
124L/2    124L/2    CHARLEROI-FAISCEAU A    77  
          noline    CHARLEROI-FAISCEAU A    41  
130       124L/2    CHARLEROI-FAISCEAU A  -146  
                    MARCINELLE            -145  
          130       COUILLET               585  
130A      130A      MARCHIENNE-ZONE       2806  
          132       JAMIOULX              1005  
130C      130C      COUILLET               579  
140/1     140       CHARLEROI-OUEST        477  
          noline    CHARLEROI-OUEST        163  
noline    124A      ROUX                     1  
          124L/2    CHARLEROI-FAISCEAU A    34  
                    MARCINELLE             136